# ncbi-text-drug-resistance

Use the "Run" button to execute the code.

In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import jovian

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="ncbi-text-drug-resistance")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "anubratadas/ncbi-text-drug-resistance" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/anubratadas/ncbi-text-drug-resistance


'https://jovian.ai/anubratadas/ncbi-text-drug-resistance'

In [5]:
import requests
from bs4 import BeautifulSoup as bs 
import time
import xml.etree.ElementTree as ET
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re

In [6]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [8]:
drugs_table=pd.read_csv('revised_db.txt',sep="|",header=0)

In [18]:
drugs_table.head()

,Product,DrugBank ID,Indications,Targets
0,Abemaciclib,"<a href=""https://go.drugbank.com/drugs/DB12001...",Advanced Breast Cancer; Metastatic Breast Cancer,CDK6; CDK4; CCND3; CDKN2A; CCND2; SMARCA4
1,Abiraterone,"<a href=""https://go.drugbank.com/drugs/DB05812...",Metastatic Castration Resistant Prostate Cancer,CYP17A1; DSE; TSPYL1
2,Acalabrutinib,"<a href=""https://go.drugbank.com/drugs/DB11703...",Chronic Lymphocytic Leukaemia (CLL); Mantle Ce...,BTK; TEC; TXK; BMX
3,Aclarubicin,"<a href=""https://go.drugbank.com/drugs/DB11617...",NaN,SMN1; TOP2B; CSF2
4,Afatinib,"<a href=""https://go.drugbank.com/drugs/DB08916...",Metastatic Non-Small Cell Lung Cancer; Refract...,EGFR; ERBB2; ERBB4; KRAS; EML4; NRG1; NRAS; ERBB3


In [ ]:
drugs_table.shape

In [9]:
drugs_list=drugs_table.Product.to_list()

In [10]:
drug_resistance_list=[drug+" AND resistance AND mutations" for drug in drugs_list]

In [17]:
len(drug_resistance_list)

281

In [19]:
# from PMC Database
url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi" # base url
def get_efetch_uids(item): 
    uid_lst=[] # list object to collect the uids generated by Esearch    
    try:
        payload={"db":"pmc","term":item,"retype":"count","retmax":2000,"email":"anubratadas@gmail.com","api_key":"9577c7078191e23dfdd5015746af31abf708" } # parameters
        response=requests.get(url,params=payload) # response object
        response.raise_for_status()
        root=ET.fromstring(response.content) # tree obtained directly from response.content using fromstring
        id_list=root.find('IdList')   
        return id_list
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    except requests.exceptions.TooManyRedirects:
        print("please try a different URL")
            
def call_efetch(drug_resistance_list): # removed "cancer AND mutations" and "cancer AND pathways AND mutations"
    term=drug_resistance_list
    drug_resistance_dict={}
    for item in term:
        uid_list=[]
        print(item)
        time.sleep(1)
        id_list=get_efetch_uids(item)  
        if id_list is not None:
            for id_num in id_list:   
                uid_list.append(id_num.text)
        drug_resistance_dict[item]=uid_list        
        print(len(uid_list))
     
    print(f' {len(uid_list)} records retrieved')
    return drug_resistance_dict
uid_dict=call_efetch(drug_resistance_list)

Abemaciclib AND resistance AND mutations
1110
Abiraterone AND resistance AND mutations
2000
Acalabrutinib AND resistance AND mutations
449
Aclarubicin AND resistance AND mutations
209
Afatinib AND resistance AND mutations
2000
Aflibercept AND resistance AND mutations
1224
Aldesleukin AND resistance AND mutations
361
Alectinib AND resistance AND mutations
1570
Alemtuzumab AND resistance AND mutations
1404
Alitretinoin AND resistance AND mutations
84
Alpelisib AND resistance AND mutations
953
Altretamine AND resistance AND mutations
59
Aminolevulinic Acid AND resistance AND mutations
1901
Amivantamab AND resistance AND mutations
40
Amrubicin AND resistance AND mutations
163
Amsacrine AND resistance AND mutations
379
Anastrozole AND resistance AND mutations
1706
Apalutamide AND resistance AND mutations
370
Arsenic Trioxide AND resistance AND mutations
2000
Asparaginase AND resistance AND mutations
2000
Atezolizumab AND resistance AND mutations
2000
Avapritinib AND resistance AND mutations

1036
Niraparib AND resistance AND mutations
1307
Nivolumab AND resistance AND mutations
2000
Obinutuzumab AND resistance AND mutations
873
Ofatumumab AND resistance AND mutations
958
Olaparib AND resistance AND mutations
2000
Olaratumab AND resistance AND mutations
169
Omacetaxine Mepesuccinate AND resistance AND mutations
649
Osimertinib AND resistance AND mutations
2000
Oxaliplatin AND resistance AND mutations
2000
Paclitaxel AND resistance AND mutations
2000
Padeliporfin AND resistance AND mutations
5
Palbociclib AND resistance AND mutations
2000
Panitumumab AND resistance AND mutations
2000
Panobinostat AND resistance AND mutations
2000
Pazopanib AND resistance AND mutations
2000
Pegaspargase AND resistance AND mutations
124
Peginterferon Alfa-2b AND resistance AND mutations
641
Pembrolizumab AND resistance AND mutations
2000
Pemetrexed AND resistance AND mutations
2000
Pemigatinib AND resistance AND mutations
147
Pentostatin AND resistance AND mutations
367
Pertuzumab AND resistan

In [33]:
with open("uid_drugs_resistance.txt","w") as fh:
    fh.write(uid)

In [27]:
# count=0
# while count <=1:
#     for key in uid_dict.keys(): 
#         print(key)
#         print(uid_dict[key])
#         count=count+1
#         break

In [7]:
import json

In [29]:
with open("drug_resistance_uid_json_file.txt","w") as jsonfh:
    j=json.dumps(uid_dict,indent=4)
    jsonfh.write(j)

In [8]:
with open("drug_resistance_uid_json_file.txt","r") as jsonfh:
    uid_data=json.load(jsonfh)

In [9]:
type(uid_data)

dict

In [ ]:
# we are creating a shorter list

In [12]:
for item in uid_data:
    if len(uid_data[item])<50:
        print(len(uid_data[item]))

40
49
45
6
27
41
17
44
42
22
27
8
43
14
5
34
3
46
35
10
19
32
49


In [ ]:
# names of approved genes obtained from HUGO site
# https://www.genenames.org/tools/search/#!/?query=&rows=20&start=0&filter=document_type:gene

In [17]:
with open("hgnc-search-1633607772666.json","r") as fh:
    gene_names=json.load(fh)

In [18]:
type(gene_names)

list

In [22]:
gene_name_symbols=[item['symbol'] for item in gene_names]

In [23]:
gene_name_symbols[:3]

['A1BG', 'A1BG-AS1', 'A1CF']

In [25]:
with open('gene_names_json.txt', 'w') as jsonfh1:
    j=json.dumps(gene_name_symbols)
    jsonfh1.write(j)